# Music Retrieval / Audio Identification

**Course:** Introduction to Information Retrieval (194.166)  
**Milestone 2.1:** Audio Processing  
**Group:** 26

| Student Name           | Student ID |
|------------------------|------------|
| Nikita Lysenko         | 12127033   |
| Yagmur Coban           | 12313405   |
| Nikolaus Winkelhofer  | 12317833   |

## Overview
Exercise 2 deals with a specific music retrieval task, namely audio identification. Milestone 2.1 covers audio processing aspects and measuring retrieval effectiveness under various query preprocessing and distortion conditions.

## 1. Obtaining Data and Query Preparation (30%)

- Original: the unmodified 10 sec extracted segment
- Noise: add noticeable (Gaussian) noise to the segment
- Coding: a highly compressed representation with clearly audible artifacts
- Mobile: play back the segments via a laptop or phone in an urban outdoor setting and record it with another device (phone). There is no need for the recording to be exactly 10 seconds, the recorded music segment can be truncated and/or surrounded by noise

## 1.1 Add Noise
used noise level 0.04.

## 1.2 Compress Track
compressed them to 16kbps

## 1.3 Play on other device and record it with microphone

# Task 2: Database Preparation (20%)

1.  **Spectrogram Generation**: Converts audio signals into the time-frequency domain using STFT.
2.  **Constellation Map Extraction**: Identifies local maxima (peaks) to create robust fingerprints.
3.  **Matching**: Compares query fingerprints against a database.
4.  **Evaluation**: Calculates detailed metrics including **Precision, Recall, F1-Score**, and **Accuracy** across different query categories (Original, Noise, Mobile, Coding).


#

In [11]:
import os, sys
import numpy as np
from numba import jit
import librosa
from scipy import ndimage
from matplotlib import pyplot as plt
import IPython.display as ipd
import time
import glob
import pickle
import pandas as pd
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

## Paths

In [12]:
BASE_PATH = "/content/drive/MyDrive/IR Exercise 2"
DB_AUDIO_PATH = os.path.join(BASE_PATH, "raw_30s_audio-26", "26")
QUERY_AUDIO_PATH = os.path.join(BASE_PATH, "music_queries")
OUTPUT_DB_FILE = os.path.join(BASE_PATH, "db_fingerprints.pkl")
OUTPUT_CSV_FILE = os.path.join(BASE_PATH, "detailed_results.csv")

## Parameters

In [13]:
PARAMETER_SETS = [
    {'id': 0, 'dist_freq': 3, 'dist_time': 3},
    # {'id': 1, 'dist_freq': 4, 'dist_time': 2},
    # {'id': 2, 'dist_freq': 6, 'dist_time': 4},
    # {'id': 3, 'dist_freq': 9,  'dist_time': 5},
    # {'id': 4, 'dist_freq': 12, 'dist_time': 6},
    # {'id': 5, 'dist_freq': 15, 'dist_time': 8},
    # {'id': 6, 'dist_freq': 25, 'dist_time': 10},
    # {'id': 7, 'dist_freq': 35, 'dist_time': 15},
    # {'id': 8, 'dist_freq': 50, 'dist_time': 20},
    # {'id': 9,  'dist_freq': 10, 'dist_time': 5},
    # {'id': 10, 'dist_freq': 10, 'dist_time': 5},
    # {'id': 11, 'dist_freq': 7,  'dist_time': 30},
]

## Optimization

In [14]:
# Optimization: Limit the search for match start positions to the first 21 seconds
MAX_SCAN_START_SEC = 21.0

## 2.1 Compute Spectrogram
Loads audio and computes the Short-Time Fourier Transform (STFT).
We used the method of the reference jupyter notebook and added a max processing limit of 30s as stated in the task description.

In [15]:
def compute_spectrogram(fn_wav, Fs=22050, N=2048, H=1024, bin_max=None, frame_max=None):
    """Computes the magnitude spectrogram of an audio file.

    Args:
        fn_wav (str): Path to the audio file.
        Fs (int): Sampling rate in Hz.
        N (int): Window size in samples.
        H (int): Hop size in samples.
        bin_max (int): Maximum frequency bin index to return.
        frame_max (int): Maximum time frame index to return.
        duration (float): Duration in seconds to be loaded.

    Returns:
        Y (np.ndarray): Magnitude spectrogram.
    """

    x, Fs = librosa.load(fn_wav, sr=Fs, duration=30.0)
    X = librosa.stft(x, n_fft=N, hop_length=H, win_length=N, window='hann')
    if bin_max is None:
        bin_max = X.shape[0]
    if frame_max is None:
        frame_max = X.shape[0]
    Y = np.abs(X[:bin_max, :frame_max])
    return Y

## 2.2 Compute Constellation Map
Uses a maximum filter to find local peaks.

In [16]:
def compute_constellation_map(Y, dist_freq=7, dist_time=7, thresh=0.01):
    """Compute constellation map (implementation using image processing)

    Args:
        Y (np.ndarray): Spectrogram (magnitude)
        dist_freq (int): Neighborhood parameter for frequency direction (kappa) (Default value = 7)
        dist_time (int): Neighborhood parameter for time direction (tau) (Default value = 7)
        thresh (float): Threshold parameter for minimal peak magnitude (Default value = 0.01)

    Returns:
        Cmap (np.ndarray): Boolean mask for peak structure (same size as Y)
    """
    result = ndimage.maximum_filter(Y, size=[2 * dist_freq + 1, 2 * dist_time + 1], mode='constant')
    Cmap = np.logical_and(Y == result, result > thresh)
    return Cmap

## 2.3 Compute Matching Function
Matches the query against a database entry.

**Optimization**: Unlike the standard implementation, this pre-calculates the query tolerance (dilation)
    for performance and returns only the metrics of the single best match, not the full
    matching curve. It also limits the search range based on MAX_SCAN_START_SEC.

In [17]:
def compute_matching_function(C_D, C_Q, tol_freq=1, tol_time=1):
    """Computes the best match between constellation maps using optimized dilation.

    Unlike the standard implementation, this pre-calculates the query tolerance (dilation)
    for performance and returns only the metrics of the single best match, not the full
    matching curve. It also limits the search range based on MAX_SCAN_START_SEC.

    Args:
        C_D (np.ndarray): Binary matrix used as database document.
        C_Q (np.ndarray): Binary matrix used as query document.
        tol_freq (int): Tolerance in frequency direction (vertical) (Default value = 1).
        tol_time (int): Tolerance in time direction (horizontal) (Default value = 1).

    Returns:
        Delta (np.array): Matching Curve. An array containing the True Positive (TP) count for every possible time shift.
        shift_max (int): Optimal Position. The time index where Delta reaches its maximum (best alignment).
        TP_max (int): Best Score. The maximum number of matching peaks found (the peak value of Delta).
    """

    L = C_D.shape[1]
    N = C_Q.shape[1]
    M = L - N
    assert M >= 0, "Query must be shorter than document"

    Fs = 22050
    H = 1024
    max_frames = int(MAX_SCAN_START_SEC * Fs / H)

    search_range = min(M + 1, max_frames)

    Delta = np.zeros(M+1)
    for m in range(search_range):
        C_D_crop = C_D[:, m:m+N]
        TP, FN, FP, C_AND = match_binary_matrices_tol(C_D_crop, C_Q,
                                                      tol_freq=tol_freq, tol_time=tol_time)
        Delta[m] = TP
    shift_max = np.argmax(Delta)
    TP_max = Delta[shift_max]

    best_C_D_crop = C_D[:, shift_max:shift_max+N]
    n_db_best = np.sum(best_C_D_crop)
    return Delta, shift_max, TP_max, n_db_best

def match_binary_matrices_tol(C_ref, C_est, tol_freq=0, tol_time=0):
    """| Compare binary matrices with tolerance
    | Note: The tolerance parameters should be smaller than the minimum distance of
      peaks (1-entries in C_ref ad C_est) to obtain meaningful TP, FN, FP values

    Notebook: C7/C7S1_AudioIdentification.ipynb

    Args:
        C_ref (np.ndarray): Binary matrix used as reference
        C_est (np.ndarray): Binary matrix used as estimation
        tol_freq (int): Tolerance in frequency direction (vertical) (Default value = 0)
        tol_time (int): Tolerance in time direction (horizontal) (Default value = 0)

    Returns:
        TP (int): True positives
        FN (int): False negatives
        FP (int): False positives
        C_AND (np.ndarray): Boolean mask of AND of C_ref and C_est (with tolerance)
    """
    assert C_ref.shape == C_est.shape, "Dimensions need to agree"
    N = np.sum(C_ref)
    M = np.sum(C_est)
    # Expand C_est with 2D-max-filter using the tolerance parameters
    C_est_max = ndimage.maximum_filter(C_est, size=(2*tol_freq+1, 2*tol_time+1),
                                       mode='constant')
    C_AND = np.logical_and(C_est_max, C_ref)
    TP = np.sum(C_AND)
    FN = N - TP
    FP = M - TP
    return TP, FN, FP, C_AND

## 2. Task 2: Database Preparation

This section processes all reference audio files. It creates constellation maps for each defined parameter set and saves them into a `pickle` file. If the database file already exists, this step is skipped to save time.

In [18]:
def run_task2():
    print("----| START TASK 2: Database Preparation |----")

    if os.path.exists(OUTPUT_DB_FILE):
        print(f"INFO: Database '{OUTPUT_DB_FILE}' already exists.")
        print("--> Loading existing database...")
        return

    extensions = ['*.mp3', '*.wav', '*.m4a']
    db_files = []
    for ext in extensions:
        db_files.extend(glob.glob(os.path.join(DB_AUDIO_PATH, ext)))

    if not db_files:
        print(f"ERROR: No audio files found in '{DB_AUDIO_PATH}'!")
        return

    print(f"Found: {len(db_files)} files. Starting processing...")

    db_fingerprints = {p['id']: {} for p in PARAMETER_SETS}


    for fpath in tqdm(db_files, desc="Indexing Database"):
        fname = os.path.basename(fpath)

        try:
            # Only the first 30 seconds are indexed
            Y = compute_spectrogram(fpath)

            for params in PARAMETER_SETS:
                pid = params['id']
                Cmap = compute_constellation_map(Y,
                                                 dist_freq=params['dist_freq'],
                                                 dist_time=params['dist_time'])
                db_fingerprints[pid][fname] = Cmap

        except Exception as e:
            tqdm.write(f"  Error with file {fname}: {e}")
            continue

    print(f"Saving database to '{OUTPUT_DB_FILE}'...")
    with open(OUTPUT_DB_FILE, 'wb') as f:
        pickle.dump(db_fingerprints, f)
    print("Task 2 completed successfully.")
run_task2()

----| START TASK 2: Database Preparation |----
INFO: Database '/content/drive/MyDrive/IR Exercise 2/db_fingerprints.pkl' already exists.
--> Loading existing database...


## 3. Task 3: Retrieval & Evaluation

In this step, we match the query files (Original, Noise, Mobile, Coding) against our database.

**Metrics Calculation per Query:**
* **TP (True Positives):** Number of matching peaks.
* **FP (False Positives):** Peaks present in the query but missing in the DB match.
* **FN (False Negatives):** Peaks present in the DB match but missing in the query.
* **Precision**: $TP / (TP + FP)$
* **Recall**: $TP / (TP + FN)$
* **F1-Score**: $2 \cdot \frac{Precision \cdot Recall}{Precision + Recall}$

The detailed results are saved to a CSV file.

In [ ]:
def run_task3():
    print("----| START TASK 3: Retrieval Experimentation |----")

    if not os.path.exists(OUTPUT_DB_FILE):
        print("Error: Database file not found. Please run Task 2 first!")
        return pd.DataFrame()

    print("Loading database...")
    with open(OUTPUT_DB_FILE, 'rb') as f:
        db_fingerprints = pickle.load(f)

    # Find query files recursively
    search_pattern = os.path.join(QUERY_AUDIO_PATH, "**", "*.*")
    query_files = glob.glob(search_pattern, recursive=True)
    query_files = [f for f in query_files if f.lower().endswith(('.wav', '.mp3', '.m4a'))]

    if not query_files:
        print(f"No queries found in '{QUERY_AUDIO_PATH}'.")
        return pd.DataFrame()

    print(f"Found {len(query_files)} queries. Starting experiments...")

    detailed_records = []

    # --- CHANGE 1: Outer Loop for Parameters ---
    for params in tqdm(PARAMETER_SETS, desc="Configurations"):
        pid = params['id']
        config_name = f"ID{pid}_df{params['dist_freq']}_dt{params['dist_time']}"

        # Replace print with tqdm.write so the bar doesn't jump
        tqdm.write(f"\n--- Testing Config: {config_name} ---")

        current_db = db_fingerprints[pid]

        # --- CHANGE 2: Inner Loop for Queries ---
        # leave=False means this bar disappears when done (cleaner interface)
        for q_path in tqdm(query_files, desc="Processing Queries", leave=False):
            q_name = os.path.basename(q_path)

            # Identify Category based on folder/filename
            q_type = 'Unknown'
            path_str = q_path.lower()
            if 'original' in path_str: q_type = 'Original'
            elif 'noise' in path_str: q_type = 'Noise'
            elif 'coding' in path_str: q_type = 'Coding'
            elif 'mobile' in path_str: q_type = 'Mobile'

            if q_type == 'Unknown': continue

            try:
                start_time = time.time()

                # 1. Process Query
                Y_q = compute_spectrogram(q_path)
                C_q = compute_constellation_map(Y_q,
                                                dist_freq=params['dist_freq'],
                                                dist_time=params['dist_time'])

                # Total peaks in Query (M) - needed for FP calculation
                M_query_sum = np.sum(C_q)

                # 2. Match against Database
                best_match_name = None
                best_TP = -1
                best_N_DB = 0
                best_shift = 0

                for db_name, C_db in current_db.items():
                    # Call optimized matching function
                    _, shift, tp, n_db = compute_matching_function(C_db, C_q, tol_freq=1, tol_time=1)

                    if tp > best_TP:
                        best_TP = tp
                        best_N_DB = n_db
                        best_shift = shift
                        best_match_name = db_name


                query_duration = time.time() - start_time

                # 3. Check Correctness (ID matching)
                is_correct = False
                if best_match_name:
                    db_id = best_match_name.split('.')[0]
                    q_id = q_name.split('.')[0]
                    is_correct = (db_id == q_id)

                # 4. Calculate Metrics
                TP = best_TP
                FN = best_N_DB - TP
                FP = M_query_sum - TP

                precision = TP / (TP + FP) if (TP + FP) > 0 else 0
                recall = TP / (TP + FN) if (TP + FN) > 0 else 0
                f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

                # 5. Store Record
                record = {
                    'Category': q_type,
                    'Query': q_name,
                    'Configuration': config_name,
                    'Matched DB File': best_match_name,
                    'TP': TP,
                    'FN': FN,
                    'FP': FP,
                    'Precision': round(precision, 4),
                    'Recall': round(recall, 4),
                    'F1 Score': round(f1_score, 4),
                    'Shift (frames)': best_shift,
                    'Query Time (s)': round(query_duration, 4),
                    'Correct': is_correct
                }
                detailed_records.append(record)

                # --- CHANGE 3: Output ---
                # Use tqdm.write() instead of print()
                status = "OK" if is_correct else "X"
                # Optional: You can comment this out if you only want to see the bar
                tqdm.write(f"  [{q_type}] {q_name[:15]} -> {best_match_name} (TP={TP}) {status}")

            except Exception as e:
                tqdm.write(f"  Error processing {q_name}: {e}")

    # Create DataFrame
    df_results = pd.DataFrame(detailed_records)

    if not df_results.empty:
        df_results.to_csv(OUTPUT_CSV_FILE, index=False)
        print(f"\nDetailed results saved to: {OUTPUT_CSV_FILE}")

    return df_results
df_results = run_task3()

----| START TASK 3: Retrieval Experimentation |----
Loading database...
Found 80 queries. Starting experiments...


Configurations:   0%|          | 0/1 [00:00<?, ?it/s]


--- Testing Config: ID0_df3_dt3 ---


Processing Queries:   0%|          | 0/80 [00:00<?, ?it/s]

In [ ]:
if not df_results.empty:
    def highlight_correct(val):
        if val:
            return 'background-color: #198754; color: white; font-weight: bold;'
        else:
            return 'background-color: #dc3545; color: white; font-weight: bold;'

    print("Detailed Results")

    styled_df = df_results.style.applymap(highlight_correct, subset=['Correct'])


    styled_df = styled_df.format({
        'Precision': '{:.4f}',
        'Recall': '{:.4f}',
        'F1 Score': '{:.4f}',
        'Query Time (s)': '{:.2f}'
    })

    display(styled_df)

    print("\nSummary: Accuracy (%)")
    pivot_acc = df_results.pivot_table(
        index='Configuration',
        columns='Category',
        values='Correct',
        aggfunc='mean'
    ) * 100


    display(pivot_acc.style.background_gradient(cmap='RdYlGn', vmin=0, vmax=100).format("{:.2f}"))

else:
    print("No results generated.")

## 4. Analysis & Results

Below are the aggregated results of the experiment.
1.  **Detailed Table (Excerpt):** Shows metrics per specific query.
2.  **Accuracy Table:** Percentage of correctly identified songs per category and parameter set.
3.  **F1-Score Table:** Average F1-Score per category.